<a href="https://colab.research.google.com/github/RafsanJany-44/Research-Wake-NREM-REM/blob/main/scratchpad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install imbalanced-learn

from imblearn.over_sampling import SMOTE

def balance(X_temp, y_temp):
  smote = SMOTE()
  X_temp, y_temp= smote.fit_resample(X_temp, y_temp)

  return pd.concat([pd.DataFrame(X_temp), pd.DataFrame(y_temp)], axis=1)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np

dataset = pd.read_csv('https://raw.githubusercontent.com/RafsanJany-44/Thesis_Project/main/All_DATA/without_sn_and_Epoch/EEG_HMC.csv')
dataset.head()
target="Sleep Stage"

In [3]:
dataset[target] = dataset[target].replace(['W'],'Wake')
dataset[target] = dataset[target].replace(['N1'],'NREM')
dataset[target] = dataset[target].replace(['N2'],'NREM')
dataset[target] = dataset[target].replace(['N3'],'NREM')
dataset[target] = dataset[target].replace(['R'],'REM')

In [4]:
dataset[target].value_counts()

classes = np.array(sorted(list(set(dataset[target]))))

In [5]:
classes

array(['NREM', 'REM', 'Wake'], dtype='<U4')

In [6]:
X = dataset.loc[:, dataset.columns!=target]
y = dataset[target]

In [7]:
#Splitting dataset into training and testing dataset
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.33,random_state=0)

In [8]:
y_train.value_counts()

NREM    48746
Wake    12964
REM     10952
Name: Sleep Stage, dtype: int64

In [9]:
new_dataset=balance(X_train,y_train)

X_train = new_dataset.loc[:, new_dataset.columns!=target]
y_train = new_dataset[target] 

In [10]:
y_train.value_counts()

NREM    48746
Wake    48746
REM     48746
Name: Sleep Stage, dtype: int64

In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score
from tqdm import tqdm
N=5000
k_range = range (2300,N+1,100)
scores={}
scores_list = []
for k in tqdm(k_range):
  xgb_classifier = xgb.XGBClassifier(n_estimators=k,random_state=0)
  xgb_classifier.fit(X_train, y_train)
  y_pred=xgb_classifier.predict(X_test)
  scores[k] = accuracy_score(y_test,y_pred)
  scores_list.append(accuracy_score(y_test,y_pred))
  print(str(k)+"/"+str(N)+" round completed......................... Accurecy: "+str(accuracy_score(y_test,y_pred)))


%matplotlib inline
import matplotlib.pyplot as plt

plt.figure(figsize = (25,10))
plt.plot(k_range,scores_list)
plt.xlabel('Value of n_estimators')
plt.ylabel ('Testing Accuracy')



print("The best n_estimators:")
best_estimator=list(scores.keys())[scores_list.index(max(scores_list))]
print(best_estimator)

  4%|▎         | 1/28 [1:08:48<30:57:42, 4128.23s/it]

2300/5000 round completed......................... Accurecy: 0.8919500405152421


  7%|▋         | 2/28 [2:20:55<30:39:31, 4245.05s/it]

2400/5000 round completed......................... Accurecy: 0.892089748246668


 11%|█         | 3/28 [3:35:34<30:13:23, 4352.14s/it]

2500/5000 round completed......................... Accurecy: 0.8928162284500825


 14%|█▍        | 4/28 [4:54:48<30:04:15, 4510.66s/it]

2600/5000 round completed......................... Accurecy: 0.893403000922071


In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score
N=30
k_range = range (1,N+1)
scores={}
scores_list = []
for k in tqdm(k_range):
  xgb_classifier = xgb.XGBClassifier(max_depth=k,random_state=0)
  xgb_classifier.fit(X_train, y_train)
  y_pred=xgb_classifier.predict(X_test)
  scores[k] = accuracy_score(y_test,y_pred)
  scores_list.append(accuracy_score(y_test,y_pred))
  print(str(k)+"/"+str(N)+" round completed......................... Accurecy: "+str(accuracy_score(y_test,y_pred)))


%matplotlib inline
import matplotlib.pyplot as plt

plt.figure(figsize = (25,10))
plt.plot(k_range,scores_list)
plt.xlabel('Value of n_estimators')
plt.ylabel ('Testing Accuracy')



print("The best depth:")
best_depth=list(scores.keys())[scores_list.index(max(scores_list))]
print(best_depth)